In [122]:
%cd /alpha/projects/wastie/code/kondrashov/delta/notes/aggregate/finale

/alpha/projects/wastie/code/kondrashov/delta/notes/aggregate/finale


In [123]:
import pandas as pd
import pickle

In [124]:
# all_results = []
# with open(f'box_union_gridsearch.pkl', 'rb') as f:
#     all_results.append(pickle.load(f))
# with open(f'box_nms_gridsearch.pkl', 'rb') as f:
#     all_results.append(pickle.load(f))
# with open(f'box_softnms_gridsearch.pkl', 'rb') as f:
#     all_results.append(pickle.load(f))
    
# train_results = {}
# for r in all_results:
#     for k, v in r.items():
#         k = ('box', *k[1:])
#         train_results[k] = v

In [125]:
with open(f'mask_trainval_gridsearch.pkl', 'rb') as f:
    train_results = pickle.load(f)

In [126]:
def sym(flag):
    return '$\checkmark$' if flag else ' '

In [127]:
def get_table(train_results, test_results, shorten=False):
    table_list = []
    
    for merger_type in ([''] if shorten else ['Union', 'NMS', 'Soft-NMS']):
        for border_filter in [False, True]:
            for threshold_filter in [False, True]:
                for conf_filter in [False, True]:
                    best_k = None
                    best_v = None
                    best_f1 = 0.0
                    for k, v in train_results.items():
                        if not border_filter and k[2]:
                            continue
                        if not threshold_filter and k[4] != 1:
                            continue
                        if not conf_filter and k[5] > k[3]:
                            continue
                        if not any([border_filter, threshold_filter, conf_filter]):
                            if k[1] != 'union':
                                continue
                        elif not k[1].startswith(merger_type.lower()):
                            continue
                        if v[0] > best_f1:
                            best_k = k
                            best_v = v
                            best_f1 = v[0]
                    if not best_k:
                        print(merger_type, border_filter, threshold_filter, conf_filter)
                    v = test_results[best_k]
                    table_list.append([
                        sym(border_filter), 
                        sym(threshold_filter), 
                        sym(conf_filter), 
                        *([] if shorten else [merger_type]),
                        round(v[0], 2), 
                        round(v[1], 2), 
                        round(v[2], 2)
                    ])
    if shorten:
        df = pd.DataFrame(table_list, columns=["B", "T", "C", "F1", "P", "R"])
    else: 
        df = pd.DataFrame(table_list, columns=["B", "T", "C", "MT", "F1", "P", "R"])
    return df

In [128]:

with open(f'mask_sparse_gridsearch.pkl', 'rb') as f:
    sparse_test_results = pickle.load(f)
sparse_table = get_table(train_results, sparse_test_results, shorten=True)

In [129]:
with open(f'mask_dense_gridsearch.pkl', 'rb') as f:
    dense_test_results = pickle.load(f)
dense_table = get_table(train_results, dense_test_results, shorten=True)

In [130]:
table = sparse_table.copy()
table['F1_2'] = dense_table['F1']
table['P_2'] = dense_table['P']
table['R_2'] = dense_table['R']

In [131]:
table

,B,T,C,F1,P,R,F1_2,P_2,R_2
0,,,,0.89,0.90,0.88,0.68,0.73,0.65
1,,,$\checkmark$,0.82,0.78,0.88,0.66,0.67,0.65
2,,$\checkmark$,,0.84,0.80,0.88,0.67,0.72,0.63
3,,$\checkmark$,$\checkmark$,0.83,0.80,0.88,0.67,0.72,0.62
4,$\checkmark$,,,0.90,0.90,0.89,0.72,0.75,0.69
5,$\checkmark$,,$\checkmark$,0.90,0.90,0.89,0.72,0.75,0.69
6,$\checkmark$,$\checkmark$,,0.90,0.92,0.88,0.70,0.81,0.62
7,$\checkmark$,$\checkmark$,$\checkmark$,0.90,0.92,0.87,0.70,0.81,0.62


In [132]:
import pandas as pd

def highlight_grouped_max_to_latex(df, group_names=None, group_sizes=None):
    """
    Convert DataFrame to LaTeX table with max values bolded in each metric group.
    
    Args:
        df: pandas DataFrame containing the data
        group_names: list of group names (e.g., ['Разреженный тест', 'Плотный тест'])
        group_sizes: list with number of metrics in each group (e.g., [3, 3] for F1,P,R in each)
    
    Returns:
        str: LaTeX formatted table with multi-column headers
    """
    # Make a copy to avoid modifying original DataFrame
    df = df.copy()
    
    # Default groups if not specified
    if group_names is None or group_sizes is None:
        group_names = ['Разреженный тест', 'Плотный тест']
        group_sizes = [3, 3]  # F1, P, R for each group
    
    # Highlight max values in each metric group
    start_col = 0
    for size in group_sizes:
        group_metrics = df.columns[start_col:start_col+size]
        
        for col in group_metrics:
            if pd.api.types.is_numeric_dtype(df[col]):
                max_val = df[col].max()
                df[col] = df[col].apply(
                    lambda x: r'\textbf{' + f'{x:.2f}' + '}' if x == max_val else f'{x:.2f}'
                )
        
        start_col += size
    
    # Generate LaTeX header with multi-column
    header = [r'\multicolumn{3}{|c|}{Компоненты}']
    start_col = 0
    for name, size in zip(group_names, group_sizes):
        header.append(r'\multicolumn{' + f'{size}' + '}{c|}{' + f'{name}' + '}')
    
    # Create LaTeX table
    latex_table = r"""
\begin{table}[h]
\centering
\renewcommand{\arraystretch}{1.2}
\begin{tabular}{|""" + 'c|' * len(df.columns) + r"""}
\hline
""" + ' & '.join(header) + r""" \\
\hline
""" + ' & '.join(df.columns) + r""" \\
\hline
"""

    # Add data rows
    for _, row in df.iterrows():
        latex_table += ' & '.join(row.astype(str)) + r""" \\
\hline
"""

    latex_table += r"""\end{tabular}
\caption{Результаты тестирования с выделением максимальных значений по группам}
\label{tab:grouped_results}
\end{table}
"""
    
    return latex_table

In [133]:
print(highlight_grouped_max_to_latex(table))


\begin{table}[h]
\centering
\renewcommand{\arraystretch}{1.2}
\begin{tabular}{|c|c|c|c|c|c|c|c|c|}
\hline
\multicolumn{3}{|c|}{Компоненты} & \multicolumn{3}{c|}{Разреженный тест} & \multicolumn{3}{c|}{Плотный тест} \\
\hline
B & T & C & F1 & P & R & F1_2 & P_2 & R_2 \\
\hline
  &   &   & 0.89 & 0.90 & 0.88 & 0.68 & 0.73 & 0.65 \\
\hline
  &   & $\checkmark$ & 0.82 & 0.78 & 0.88 & 0.66 & 0.67 & 0.65 \\
\hline
  & $\checkmark$ &   & 0.84 & 0.80 & 0.88 & 0.67 & 0.72 & 0.63 \\
\hline
  & $\checkmark$ & $\checkmark$ & 0.83 & 0.80 & 0.88 & 0.67 & 0.72 & 0.62 \\
\hline
$\checkmark$ &   &   & \textbf{0.90} & 0.90 & \textbf{0.89} & 0.72 & 0.75 & 0.69 \\
\hline
$\checkmark$ &   & $\checkmark$ & \textbf{0.90} & 0.90 & \textbf{0.89} & 0.72 & 0.75 & 0.69 \\
\hline
$\checkmark$ & $\checkmark$ &   & \textbf{0.90} & \textbf{0.92} & 0.88 & 0.7 & 0.81 & 0.62 \\
\hline
$\checkmark$ & $\checkmark$ & $\checkmark$ & \textbf{0.90} & \textbf{0.92} & 0.87 & 0.7 & 0.81 & 0.62 \\
\hline
\end{tabular}
\caption{Р

In [134]:
dense_full_table = get_table(train_results, dense_test_results)
sparse_full_table = get_table(train_results, sparse_test_results)
table = dense_full_table.copy()
table['F1_2'] = dense_full_table['F1']
table['P_2'] = dense_full_table['P']
table['R_2'] = dense_full_table['R']

In [135]:
table

,B,T,C,MT,F1,P,R,F1_2,P_2,R_2
0,,,,Union,0.68,0.73,0.65,0.68,0.73,0.65
1,,,$\checkmark$,Union,0.68,0.73,0.65,0.68,0.73,0.65
2,,$\checkmark$,,Union,0.68,0.73,0.65,0.68,0.73,0.65
3,,$\checkmark$,$\checkmark$,Union,0.68,0.73,0.65,0.68,0.73,0.65
4,$\checkmark$,,,Union,0.71,0.77,0.65,0.71,0.77,0.65
5,$\checkmark$,,$\checkmark$,Union,0.71,0.77,0.65,0.71,0.77,0.65
6,$\checkmark$,$\checkmark$,,Union,0.70,0.81,0.61,0.70,0.81,0.61
7,$\checkmark$,$\checkmark$,$\checkmark$,Union,0.70,0.81,0.61,0.70,0.81,0.61
8,,,,NMS,0.68,0.73,0.65,0.68,0.73,0.65
9,,,$\checkmark$,NMS,0.59,0.51,0.69,0.59,0.51,0.69


In [163]:
def get_results(test_results):
    table_list = []
    for merger_type in ['Union', 'NMS', 'Soft-NMS']:
        best_test_k = None
        best_test_v = None
        best_test_f1 = 0.0
        for border_filter in [False, True]:
            for threshold_filter in [False, True]:
                for conf_filter in [False, True]:
                    best_k = None
                    best_v = None
                    best_f1 = 0.0
                    for k, v in train_results.items():
                        if not border_filter and k[2]:
                            continue
                        if not threshold_filter and k[4] != 1:
                            continue
                        if not conf_filter and k[5] > k[3]:
                            continue
                        if not k[1].startswith(merger_type.lower()):
                            continue
                        if v[0] > best_f1:
                            best_k = k
                            best_v = v
                            best_f1 = v[0]
                    v = test_results[best_k]
                    if best_test_f1 < v[0]:
                        best_test_f1 = v[0]
                        best_test_v = v
                        best_test_k = best_k
        table_list.append([
            merger_type,
            round(best_test_v[0], 2), 
            round(best_test_v[1], 2), 
            round(best_test_v[2], 2)
        ])
    df = pd.DataFrame(table_list, columns=["MT", "F1", "P", "R"])
    return df

In [164]:
get_results(sparse_test_results)

,MT,F1,P,R
0,Union,0.92,0.94,0.90
1,NMS,0.91,0.92,0.90
2,Soft-NMS,0.91,0.93,0.89


In [165]:
get_results(dense_test_results)

,MT,F1,P,R
0,Union,0.73,0.80,0.67
1,NMS,0.74,0.78,0.70
2,Soft-NMS,0.75,0.78,0.71
